In [50]:
# pip install requests
import requests
import pandas as pd
import json
import time
from datetime import datetime, timedelta


In [51]:
# City of Melbourne token
headers = {
    "key_com" : "8272f3e3cf855ca3006bd9d38e135f06a0714adb8519bf503af526af"
}

In [52]:
# Real-time: On-street parking bay sensors

# On-street parking bay sensors
onstreet_pbs_url = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/on-street-parking-bay-sensors/records"

headers = {
    "key_com" : "8272f3e3cf855ca3006bd9d38e135f06a0714adb8519bf503af526af"
}
# record limit per iteration
record_limit = 100


# loop interval: 2 min.
loop_interval = 10

# initialise current time - buffer
current_time_buffer = (datetime.now() - timedelta(seconds=20)).isoformat()


while True:
    # initialise list for storing updated_data
    updated_lst = []
    offset = 0
    while True:
        condition = {"limit" : record_limit,
                    "offset" : offset,
                    "where" : f"lastupdated > '{current_time_buffer}'"}
        
        # Get request
        onstreet_pbs_response = requests.get(onstreet_pbs_url,
                                            headers=headers,
                                            params=condition)
        # print test
        test = onstreet_pbs_response.json()
        print(test)

        # HTTP breakout condition
        res_status_cd = onstreet_pbs_response.status_code
        if res_status_cd != 200:
            if res_status_cd >= 500:
                print(f"Failed: {res_status_cd} Server Error")
            else:
                print(f"Failed: {res_status_cd} Request Error", onstreet_pbs_response.text)
            break

        # Get response into record, csv format
        record_json = onstreet_pbs_response.json()
        record_onstreet_pbs = record_json.get("results", [])
        print(f"Offset {offset}, fetched {len(record_onstreet_pbs)}")

        # break condition if no record 
        if not record_onstreet_pbs:
            break
        
        # append new record into list
        updated_lst.extend(record_onstreet_pbs)

        # update offest with record_limit
        offset += record_limit

    if updated_lst:
        # convert json into pandas dataframe for data mapping
        df_onstreet_pbs = pd.DataFrame(updated_lst)
        print(f"{len(df_onstreet_pbs)} records updated.")

        # update current time
        current_time_buffer = max(pd.to_datetime(df_onstreet_pbs["lastupdated"]))
    
    else:
        print("No record updates.")
    
    # loop report
    time.sleep(loop_interval)

print(df_onstreet_pbs)

{'total_count': 0, 'results': []}
Offset 0, fetched 0
No record updates.


KeyboardInterrupt: 

In [53]:
# 1 (Get 100 records) On-street parking bay sensors
onstreet_pbs_url = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/on-street-parking-bay-sensors/records?limit=100"

# Get request
onstreet_pbs_response = requests.get(onstreet_pbs_url, headers=headers)

# Output
onstreet_pbs_data = onstreet_pbs_response.json()

In [ ]:
# extract record list
record_onstreet_pbs = onstreet_pbs_data["results"]

# convert json to dataframe
df_onstreet_pbs = pd.json_normalize(record_onstreet_pbs)

# rename column header
df_onstreet_pbs = df_onstreet_pbs.rename(
    columns={"zone_number" : "parkingzone",
             "location.lon" : "longitude",
             "location.lat" : "latitude"}
    )

# convert datetime columns
df_onstreet_pbs["lastupdated"] = pd.to_datetime(df_onstreet_pbs["lastupdated"])
# df_onstreet_pbs["lastupdated"] = df_onstreet_pbs["lastupdated"].dt.strftime("%H:%M:%S")

df_onstreet_pbs["status_timestamp"] = pd.to_datetime(df_onstreet_pbs["status_timestamp"])
# df_onstreet_pbs["status_timestamp"] = df_onstreet_pbs["status_timestamp"].dt.strftime("%H:%M:%S")
df_onstreet_pbs["parking_date"] = df_onstreet_pbs["status_timestamp"].dt.date
df_onstreet_pbs["parking_time"] = df_onstreet_pbs["status_timestamp"].dt.time

df_onstreet_pbs["parkingzone"] = df_onstreet_pbs["parkingzone"].astype("Int64")


def parking_status(x):
    """
    Function convert parking status.
    Parameter: x is parking bay sensor record from status_description
    Returns:
    1 if parking is available (Unoccupied),
    0 if parking is not available (Present)
    """
    if x["status_description"] == "Unoccupied":
        return 1
    else:
        return 0

df_onstreet_pbs["is_available"] = df_onstreet_pbs.apply(parking_status, axis=1)

# output
print(df_onstreet_pbs.head())
print(df_onstreet_pbs.dtypes)
df_onstreet_pbs

                lastupdated          status_timestamp  parkingzone  \
0 2024-12-30 00:44:37+00:00 2024-08-18 08:23:46+00:00         7394   
1 2025-01-10 02:44:36+00:00 2024-08-14 09:22:05+00:00         7392   
2 2024-12-04 23:44:37+00:00 2024-11-28 02:30:57+00:00         7084   
3 2024-12-04 23:44:37+00:00 2024-11-27 23:23:40+00:00         7084   
4 2025-01-09 05:44:36+00:00 2023-08-21 00:09:03+00:00         7800   

  status_description  kerbsideid   longitude   latitude parking_date  \
0         Unoccupied        9344  144.959161 -37.804944   2024-08-18   
1         Unoccupied        9373  144.958363 -37.803291   2024-08-14   
2            Present        8735  144.961205 -37.802233   2024-11-28   
3         Unoccupied        8749  144.961851 -37.802304   2024-11-27   
4            Present       24505  144.957599 -37.797563   2023-08-21   

  parking_time  is_available  
0     08:23:46             1  
1     09:22:05             1  
2     02:30:57             0  
3     23:23:40        

In [ ]:
# 2. Sign plates located in each parking zone
sign_plates_loc_url = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/sign-plates-located-in-each-parking-zone/records?limit=100"

# Get request
sign_plates_loc_response = requests.get(sign_plates_loc_url, headers=headers)

# Output
sign_plates_loc_data = sign_plates_loc_response.json()

In [ ]:
# extract record list
record_sign_plates_loc = sign_plates_loc_data["results"]

# convert json to dataframe
df_sign_plates_loc = pd.json_normalize(record_sign_plates_loc)

# convert datetime columns
df_sign_plates_loc["time_restrictions_start"] = pd.to_datetime(df_sign_plates_loc["time_restrictions_start"])
df_sign_plates_loc["time_restrictions_start"] = df_sign_plates_loc["time_restrictions_start"].dt.time

df_sign_plates_loc["time_restrictions_finish"] = pd.to_datetime(df_sign_plates_loc["time_restrictions_finish"])
df_sign_plates_loc["time_restrictions_finish"] = df_sign_plates_loc["time_restrictions_finish"].dt.time

# for checking
print(df_sign_plates_loc)

print(df_sign_plates_loc["restriction_display"].unique())

# # Day of week dict {day:idx}
# day_dict = {
#     ""
# }

    parkingzone restriction_days time_restrictions_start  \
0          7003          Mon-Fri                07:30:00   
1          7009              Sat                07:30:00   
2          7056              Sat                07:30:00   
3          7066          Mon-Fri                07:30:00   
4          7062              Sat                07:30:00   
..          ...              ...                     ...   
95         7870          Mon-Fri                19:00:00   
96         7873          Mon-Fri                19:00:00   
97         7873          Sat-Sun                07:00:00   
98         7874          Sat-Sun                07:00:00   
99         7877          Mon-Sat                07:00:00   

   time_restrictions_finish restriction_display  
0                  18:30:00                  2P  
1                  12:30:00                  1P  
2                  12:30:00                  1P  
3                  18:30:00                  2P  
4                  12:30:00  

/var/folders/75/z6cm1pgd70n51s73mxk7qh480000gn/T/ipykernel_1693/1632403442.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_sign_plates_loc["time_restrictions_start"] = pd.to_datetime(df_sign_plates_loc["time_restrictions_start"])
/var/folders/75/z6cm1pgd70n51s73mxk7qh480000gn/T/ipykernel_1693/1632403442.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_sign_plates_loc["time_restrictions_finish"] = pd.to_datetime(df_sign_plates_loc["time_restrictions_finish"])


In [ ]:
# 2. (Static record) Sign plates located in each parking zone
with open("Dataset/sign-plates-located-in-each-parking-zone.json", "r") as file:
    data_sign_plates = json.load(file)

# convert json to dataframe
df_sign_plates = pd.json_normalize(data_sign_plates)

# format column data type
df_sign_plates["time_restrictions_start"] = pd.to_datetime(df_sign_plates["time_restrictions_start"], format="%H:%M:%S")

df_sign_plates["time_restrictions_finish"] = pd.to_datetime(df_sign_plates["time_restrictions_finish"], format="%H:%M:%S")

# output
print(df_sign_plates.head())
print(df_sign_plates.dtypes)

   parkingzone restriction_days time_restrictions_start  \
0         7033          Mon-Fri     1900-01-01 07:30:00   
1         7047              Sat     1900-01-01 07:30:00   
2         7068              Sat     1900-01-01 07:30:00   
3         7089              Sat     1900-01-01 07:30:00   
4         7109              Sat     1900-01-01 07:30:00   

  time_restrictions_finish restriction_display  
0      1900-01-01 18:30:00                  2P  
1      1900-01-01 12:30:00                  2P  
2      1900-01-01 12:30:00                  1P  
3      1900-01-01 12:30:00                  2P  
4      1900-01-01 12:30:00                  1P  
parkingzone                          int64
restriction_days                    object
time_restrictions_start     datetime64[ns]
time_restrictions_finish    datetime64[ns]
restriction_display                 object
dtype: object


In [ ]:
count_dist= df_sign_plates["parkingzone"].unique()
pd.value_counts(count_dist)

/var/folders/75/z6cm1pgd70n51s73mxk7qh480000gn/T/ipykernel_3740/68085418.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(count_dist)


7033    1
7781    1
7659    1
7677    1
7695    1
       ..
7802    1
7818    1
7814    1
7819    1
7806    1
Name: count, Length: 757, dtype: int64

In [ ]:
# 3. Parking zone linked to street segments
parking_zone_url = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/parking-zones-linked-to-street-segments/records?limit=100"

# Get request
parking_zone_response = requests.get(parking_zone_url, headers=headers)

# Output
parking_zone_data = parking_zone_response.json()

In [ ]:
# extract record list
record_parking_zone = parking_zone_data["results"]

# convert json to dataframe
record_parking_zone = pd.json_normalize(record_parking_zone)

print(record_parking_zone)
print(record_parking_zone.dtypes)

    parkingzone          onstreet            streetfrom            streetto  \
0          7000       Poplar Road       Upfield Railway      Kendall Avenue   
1          7031   Cardigan Street    Argyle Place North      Grattan Street   
2          7068      Lygon Street        Faraday Street        Elgin Street   
3          7067      Lygon Street         Pelham Street  Argyle Place North   
4          7118   Swanston Street  Lincoln Square North      Grattan Street   
..          ...               ...                   ...                 ...   
95         7229    Village Street        Collins Street       McCrae Street   
96         7236     Albert Street        Morrison Place    Lansdowne Street   
97         7258  Lansdowne Street       Cathedral Place       Albert Street   
98         7260  Lansdowne Street         Albert Street     Victoria Parade   
99         7284   Victoria Parade      Nicholson Street     Gisborne Street   

    segment_id  
0        22405  
1        20512  


In [ ]:
# 3. (Static record) Parking zone linked to street segments
with open("Dataset/parking-zones-linked-to-street-segments.json", "r") as file:
    data_parking_zones = json.load(file)

df_parking_zones = pd.json_normalize(data_parking_zones)
print(df_parking_zones)


     parkingzone         onstreet            streetfrom            streetto  \
0           7000      Poplar Road       Upfield Railway      Kendall Avenue   
1           7031  Cardigan Street    Argyle Place North      Grattan Street   
2           7068     Lygon Street        Faraday Street        Elgin Street   
3           7067     Lygon Street         Pelham Street  Argyle Place North   
4           7118  Swanston Street  Lincoln Square North      Grattan Street   
..           ...              ...                   ...                 ...   
793         7963   Rosslyn Street         Howard Street         King Street   
794         7950    Railway Place        Stanley Street        Roden Street   
795         7968   Spencer Street       La Trobe Street     Jeffcott Street   
796         7975   Stanley Street           King Street      Spencer Street   
797         7995   William Street          Capel Street      Rosslyn Street   

     segment_id  
0         22405  
1         20512

In [ ]:
# (static) 4. On-street parking bays
with open("Dataset/on-street-parking-bays.json", "r") as file:
    data_parking_bays = json.load(file)

# convert json to dataframe
df_parking_bays = pd.json_normalize(data_parking_bays)

# rename column
df_parking_bays = df_parking_bays.rename(columns={"roadsegmentid" : "segment_id"})

# format column data type
df_parking_bays["kerbsideid"] = pd.to_numeric(df_parking_bays["kerbsideid"], errors="coerce")

df_parking_bays["lastupdated"] = pd.to_datetime(df_parking_bays["lastupdated"], format="%Y-%m-%d")

print(df_parking_bays.head())
print(df_parking_bays.dtypes)

   segment_id  kerbsideid                             roadsegmentdescription  \
0       21376         NaN  Eades Place between Stanley Street and Roden S...   
1       21376         NaN  Eades Place between Stanley Street and Roden S...   
2       22651     53514.0  Elizabeth Street between Little La Trobe Stree...   
3       21957         NaN  Vale Street South between Punt Road and Vale S...   
4       21346         NaN  Curzon Street between Miller Street and Victor...   

    latitude   longitude lastupdated  location.lon  location.lat  
0 -37.806315  144.951000  2023-10-31    144.951000    -37.806315  
1 -37.806336  144.951010  2023-10-31    144.951010    -37.806336  
2 -37.809539  144.961113  2024-08-02    144.961113    -37.809539  
3 -37.819311  144.989214  2024-08-08    144.989214    -37.819311  
4 -37.805509  144.947801  2023-10-31    144.947801    -37.805509  
segment_id                         int64
kerbsideid                       float64
roadsegmentdescription            o

In [ ]:
# this merged only work when their is enough record available
df_static_merged = (
    df_parking_bays.merge(df_parking_zones, on="segment_id", how="right")
    .merge(df_sign_plates, on="parkingzone", how="right")
)

print(df_static_merged)

       segment_id  kerbsideid  \
0         20508.0         NaN   
1         20508.0         NaN   
2         20508.0         NaN   
3         20508.0         NaN   
4         20508.0         NaN   
...           ...         ...   
54992         NaN         NaN   
54993         NaN         NaN   
54994         NaN         NaN   
54995         NaN         NaN   
54996         NaN         NaN   

                                  roadsegmentdescription   latitude  \
0      Cardigan Street between Argyle Place South and... -37.803143   
1      Cardigan Street between Argyle Place South and... -37.803119   
2      Cardigan Street between Argyle Place South and... -37.803176   
3      Cardigan Street between Argyle Place South and... -37.802873   
4      Cardigan Street between Argyle Place South and... -37.802926   
...                                                  ...        ...   
54992                                                NaN        NaN   
54993                              

In [ ]:
# this merge only contain 8 record
parking_merge1 = pd.merge(
    df_onstreet_pbs,
    record_parking_zone,
    left_on=["parkingzone"],
    right_on=["parkingzone"],
    how="inner"
)

print(parking_merge1)

                lastupdated          status_timestamp  parkingzone  \
0 2025-01-21 03:42:37+00:00 2025-01-20 05:57:00+00:00         7053   
1 2025-01-21 03:42:37+00:00 2025-01-21 02:13:44+00:00         7053   
2 2025-01-21 03:42:37+00:00 2025-01-16 04:56:56+00:00         7053   
3 2025-01-21 03:42:37+00:00 2025-01-18 04:03:08+00:00         7053   
4 2025-01-21 03:42:37+00:00 2025-01-10 07:20:16+00:00         7053   
5 2025-01-21 03:42:37+00:00 2025-01-17 20:39:32+00:00         7053   
6 2024-12-11 04:44:36+00:00 2024-12-06 01:42:44+00:00         7018   
7 2025-06-23 05:44:34+00:00 2025-02-28 00:25:04+00:00         7922   

  status_description  kerbsideid   longitude   latitude parking_date  \
0            Present       10280  144.972402 -37.795027   2025-01-20   
1         Unoccupied       10287  144.971412 -37.794919   2025-01-21   
2            Present       10283  144.971971 -37.794980   2025-01-16   
3            Present       24369  144.972718 -37.795062   2025-01-18   
4        

In [ ]:
# !pip install sqlalchemy psycopg2-binary

# connect with aws database
from sqlalchemy import create_engine

engine = create_engine()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.6 MB/s eta 0:00:0000:0100:01


In [ ]:
# check connection
with engine.connect() as conn:
    result = conn.execute("SELECT NOW()")
    print(result.fetchone())